In [ ]:
import cv2
import mediapipe as mp
import torch
import numpy as np
from ultralytics import YOLO

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# Load YOLOv8 model
model_path = "yolov8n.pt"  # Ensure you have the trained model
model = YOLO(model_path)

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # YOLOv8 face detection
    results = model(frame)
    detections = results[0].boxes.xyxy.cpu().numpy()
    
    for det in detections:
        x1, y1, x2, y2 = map(int, det)
        face_roi = frame[y1:y2, x1:x2]
        
        # Convert to RGB for MediaPipe
        rgb_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_face)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                for landmark in face_landmarks.landmark:
                    h, w, _ = face_roi.shape
                    x, y = int(landmark.x * w) + x1, int(landmark.y * h) + y1
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    # Display output
    cv2.imshow("Face Profiling", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
